In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 11.1 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

df_global = pd.DataFrame()
for t in ['YPFD.BA', 'GGAL.BA', 'PAMP.BA', 'TGNO4.BA', 'SUPV.BA']:
  data = yf.download(  # or pdr.get_data_yahoo(...
          tickers = t,
          period = "ytd",
          interval = "1h",
          group_by = 'ticker',
          auto_adjust = True,
          prepost = True,
          threads = True,
          proxy = None
      )

  data['Date'] = data.index
  data['Ticker'] = t
  data['Close_-3hs'] = data.Close.shift(3)
  data['Close_+3hs'] = data.Close.shift(-3)
  data['first_3_hours'] = np.where(data['Close'] > data['Close_-3hs'] * 1.01, 'A', np.where(data['Close'] > data['Close_-3hs'], 'M', 'B'))
  data['last_3_hours'] = np.where(data['Close'] * 1.01 < data['Close_+3hs'], 'A', np.where(data['Close'] < data['Close_+3hs'], 'M', 'B'))
  #data[data.Date.dt.hour == 13].dropna().groupby(['first_3_hours', 'last_3_hours']).size().unstack()
  data['Amplitud_hora'] = (data['High'] - data['Low'])/data['Close']
  data['Volumen_Ascendente'] = np.where(data['Volume'] > data['Volume'].shift(1), 1, 0)
  data['Variacion_hora'] = data.Close / data.Close.shift(1) - 1
  data['Amplitud_hora_1l'] = data['Amplitud_hora'].shift(1)
  data['Amplitud_hora_2l'] = data['Amplitud_hora'].shift(2)
  data['Variacion_hora_1l'] = data['Variacion_hora'].shift(1)
  data['Variacion_hora_2l'] = data['Variacion_hora'].shift(2)
  data['Target'] = np.where(data['last_3_hours'] == 'A', 1, 0)
  data = data[(data.Date.dt.hour == 13)&(data.first_3_hours == 'A')].dropna().reset_index(drop=True)[['Ticker', 'Date', 'Amplitud_hora','Volumen_Ascendente','Variacion_hora','Amplitud_hora_1l','Amplitud_hora_2l','Variacion_hora_1l','Variacion_hora_2l', 'Target']]
  df_global = df_global.append(data, ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
X = data[['Amplitud_hora','Volumen_Ascendente','Variacion_hora','Amplitud_hora_1l','Amplitud_hora_2l','Variacion_hora_1l','Variacion_hora_2l']]
y = data.Target
from sklearn.model_selection import train_test_split
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(X, y, test_size = 0.3)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_training_data, y_training_data)
predictions = model.predict(x_test_data)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test_data, predictions))


[[16  0]
 [ 7  0]]
